# Time Series Analysis Gold - Data Extraction
This workbook will  be used to extract the data needed for the time series analysis.

Env: TimeSeries (Python3.9.15)<br>
Created By: TW<br>
Created On: 2023-02-23

## Libraries

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import yfinance as yf

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt




## Extract the Data
Obtain the gold prices for as far back as possible using yfinanace. Hashed out code to maintain csv.

In [21]:
gold = yf.download("GC=F",'2000-09-01', '2023-02-01')
gold

#hashed out not to change dataset

#gold.to_csv("C:/Users/twingers/OneDrive/Documents/python_projects/Time_Series_Analysis_Gold/data/gold_prices.csv")

[*********************100%***********************]  1 of 1 completed


In [22]:
dow = yf.download('^DJI', '2000-09-01', '2023-02-01')
display(dow)

# hashed out not to change dataset

dow.to_csv("C:/Users/twingers/OneDrive/Documents/python_projects/Time_Series_Analysis_Gold/data/dji.csv")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-09-01,11219.540039,11313.599609,11219.540039,11238.780273,11238.780273,125140000
2000-09-05,11221.759766,11301.669922,11184.769531,11260.610352,11260.610352,174500000
2000-09-06,11253.580078,11401.190430,11253.209961,11310.639648,11310.639648,224760000
2000-09-07,11316.009766,11323.870117,11223.240234,11259.870117,11259.870117,222220000
2000-09-08,11261.719727,11261.719727,11191.049805,11220.650391,11220.650391,181620000
...,...,...,...,...,...,...
2023-01-25,33538.359375,33773.089844,33273.210938,33743.839844,33743.839844,341720000
2023-01-26,33771.660156,33953.789062,33635.601562,33949.410156,33949.410156,327000000
2023-01-27,33952.929688,34164.328125,33830.851562,33978.078125,33978.078125,441260000


Use Selenium and BS4 to scrape the CPI Index

In [24]:
# Initiate the web driver 
driver = webdriver.Chrome('C:/Users/twingers/chromedriver_win32/chromedriver.exe')

# Load URL location to driver 
url = 'https://www.usinflationcalculator.com/inflation/consumer-price-index-and-annual-percent-changes-from-1913-to-2008/'

driver.get(url)

# Wait for the page to fully load
driver.implicitly_wait(5)

#scrape table with BS4
soup = BeautifulSoup(driver.page_source, 'lxml')
tables = soup.find_all('table')

#load in Pandas
tabl = pd.read_html(str(tables))

driver.close()


In [25]:
#clean up unneeded rows and columns
df = pd.DataFrame(tabl[0])
df.drop(labels=[13, 14, 15, 16], inplace=True, axis=1)
df.columns= df.iloc[1]
df.drop([0,1], inplace=True, axis=0)
df.rename(columns={'June': 'Jun', 'July':'Jul'}, inplace=True)
df

1,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
2,1913,9.8,9.8,9.8,9.8,9.7,9.8,9.9,9.9,10.0,10.0,10.1,10.0
3,1914,10.0,9.9,9.9,9.8,9.9,9.9,10.0,10.2,10.2,10.1,10.2,10.1
4,1915,10.1,10.0,9.9,10.0,10.1,10.1,10.1,10.1,10.1,10.2,10.3,10.3
5,1916,10.4,10.4,10.5,10.6,10.7,10.8,10.8,10.9,11.1,11.3,11.5,11.6
6,1917,11.7,12.0,12.0,12.6,12.8,13.0,12.8,13.0,13.3,13.5,13.5,13.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,2019,251.712,252.776,254.202,255.548,256.092,256.143,256.571,256.558,256.759,257.346,257.208,256.974
109,2020,257.971,258.678,258.115,256.389,256.394,257.797,259.101,259.918,260.280,260.388,260.229,260.474
110,2021,261.582,263.014,264.877,267.054,269.195,271.696,273.003,273.567,274.310,276.589,277.948,278.802
111,2022,281.148,283.716,287.504,289.109,292.296,296.311,296.276,296.171,296.808,298.012,297.711,296.797


Convert table with year rows and month columns to table with date column

In [26]:
df2 = df.copy()
df2 = pd.melt(df, id_vars=["Year"], var_name="Month", value_name = "CPI")
df2.head()

,Year,Month,CPI
0,1913,Jan,9.8
1,1914,Jan,10.0
2,1915,Jan,10.1
3,1916,Jan,10.4
4,1917,Jan,11.7


In [27]:
df2['Month'] = df2.Month + " " + df2.Year.map(str)
df2 = df2.drop('Year', axis=1)
df2["Month"] = pd.to_datetime(df2.Month, format="%b %Y", dayfirst=True)

df2 = df2.sort_values(by="Month")
df2 = df2.set_index("Month")
df2


,CPI
Month,
1913-01-01,9.8
1913-02-01,9.8
1913-03-01,9.8
1913-04-01,9.8
1913-05-01,9.7
...,...
2023-08-01,NaN
2023-09-01,NaN
2023-10-01,NaN


In [28]:
df2.tail(12)
# TODO remove the unused data

,CPI
Month,
2023-01-01,299.170
2023-02-01,Avail. March 14
2023-03-01,NaN
2023-04-01,NaN
2023-05-01,NaN
2023-06-01,NaN
2023-07-01,NaN
2023-08-01,NaN
2023-09-01,NaN


In [29]:
df_cpi = df2.iloc[:-11]
df_cpi.tail()

,CPI
Month,
2022-09-01,296.808
2022-10-01,298.012
2022-11-01,297.711
2022-12-01,296.797
2023-01-01,299.170


In [30]:
#hashed out not to change dataset

#df_cpi.to_csv("C:/Users/twingers/OneDrive/Documents/python_projects/Time_Series_Analysis_Gold/data/cpi.csv")